# Challenge setup geth

It is assumed that the server/geth node as well as the smartcode client docker containers run on the same host. 

## Start geth

TODO



## Connect and check chain state

In [1]:
!pwd

/smartcode/notebooks/challenges


In [2]:
!ls 

admin.ipynb  setup.ipynb  test-live.ipynb  test.ipynb


In [182]:
import sys
import os
import requests
import json
import csv
import decimal
import copy

#sys.path.append("/smartcode/util")
import util
from importlib import reload
reload(util)

<module 'util' from '/smartcode/notebooks/challenges/util/__init__.py'>

### Connect and check state

Also note that for *PoA* nodes the middleware needs to be added.
Therefore, set the `poa` flag to true when connecting. 

In [230]:
#w3 = web3.Web3(web3.Web3.HTTPProvider("http://172.18.0.1:8545"))
# check if connection was successful
w3=util.connect(host="172.18.0.5",port="8544",poa=True) 
#w3=util.connect(host="172.18.0.4",port="8545",poa=True)  
#w3=util.connect(host="172.18.0.5",port="8545",poa=True)  
assert w3.isConnected()

In [71]:
w3.api #web3 version used

'5.13.0'

In [72]:
w3.clientVersion # remote client version to which w3 is connected 

'Geth/alice/v1.9.23-stable-8c2f2715/linux-amd64/go1.15.4'

In [73]:
assert w3.net.version == "1337"
w3.net.version

'1337'

In [74]:
w3.eth.blockNumber

18534

In [75]:
w3.eth.getBlock(w3.eth.blockNumber)["gasLimit"]

8000000

In [76]:
w3.eth.mining

True

In [77]:
w3.eth.syncing

False

#### Networ check

In [78]:
w3.net.peerCount

1

In [79]:
w3.geth.admin.peers()

[AttributeDict({'enode': 'enode://b9d3454d68bdada0af01ab1ab88eda014ed4b753b4233de035797d545d0754541016819f0f8ddcad8b1f50c154a349b5a78c30c31ced4c1c78edd3b636fe1d1d@172.18.0.1:32964',
  'id': '658babd677bce1894690e303853eb336cd9ae23ea3b39cab596e01c893a39af0',
  'name': 'Geth/bob/v1.9.23-stable-8c2f2715/linux-amd64/go1.15.4',
  'caps': ['eth/63', 'eth/64', 'eth/65'],
  'network': AttributeDict({'localAddress': '172.18.0.5:30303',
   'remoteAddress': '172.18.0.1:32964',
   'inbound': True,
   'trusted': False,
   'static': False}),
  'protocols': AttributeDict({'eth': AttributeDict({'version': 65,
    'difficulty': 1,
    'head': '0xdede33d41529ddf39473ee748e66921e107aa6431db6fc2481dbcef833c583e4'})})})]

In [80]:
w3.net.listening

True

In [81]:
w3.eth.accounts

['0xe63419B7c0be62A6127923fb70C32A5ca1926a16']

In [82]:
if w3.clientVersion.startswith('Parity'):
    enode = w3.parity.enode()
elif w3.clientVersion.startswith('Geth'):
    enode = w3.geth.admin.nodeInfo()['enode']
enode

'enode://4f840bf9e6db654e7930a38aca6d1870c3a6a28ddc62aa4bd04c1703455404ec3ff120b357eafa013fd2d05cf3ea31d7f6fa1d27ff4f0bfaab2d9dd2b87d1bba@127.0.0.1:30303?discport=0'

### export (correct) enod einfo for server docker container 

In [91]:
!cd /smartcode && pwd && ls

/smartcode
LICENSE			   geth_init_bob.sh
README.md		   geth_run_alice.sh
challenges		   geth_run_bob.sh
datadir			   go-ethereum
docker_build_ganache.sh    logs
docker_build_smartcode.sh  notebooks
docker_build_smartenv.sh   openethereum
docker_debug_smartcode.sh  participants
docker_debug_smartenv.sh   passwordfile
docker_exec_smartcode.sh   smartcode.python.latest.Dockerfile
docker_run_ganache.sh	   smartcode.python.requirements.txt
docker_run_smartcode.sh    smartenv.geth.latest.Dockerfile
general_config		   smartenv.openethereum.latest.Dockerfile
genesis_config		   smartenv.python.requirements.txt
geth_account.sh		   solutions
geth_attach.sh		   util
geth_init_alice.sh


In [114]:
!cd /smartcode && mkdir -p ./generated_config

In [115]:
invalid_ip_port = "172.18.0.5:30303"
valid_ip_port = "128.131.169.35:30303"
enode_file = "./generated_config/enodes"

enode_invalid = enode
enode_valid = enode_invalid.replace(invalid_ip_port,valid_ip_port)
print(enode_valid)

!cd /smartcode && echo $enode_valid > $enode_file

enode://4f840bf9e6db654e7930a38aca6d1870c3a6a28ddc62aa4bd04c1703455404ec3ff120b357eafa013fd2d05cf3ea31d7f6fa1d27ff4f0bfaab2d9dd2b87d1bba@127.0.0.1:30303?discport=0


In [116]:
# block 0 ist the genesis block
genesis_block = w3.eth.getBlock(0)
genesis_block

AttributeDict({'difficulty': 1,
 'proofOfAuthorityData': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000e63419b7c0be62a6127923fb70c32a5ca1926a160000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'gasLimit': 15000000,
 'gasUsed': 0,
 'hash': HexBytes('0xdede33d41529ddf39473ee748e66921e107aa6431db6fc2481dbcef833c583e4'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'miner': '0x00000000000

## Import students

In [117]:
!ls /smartcode/participants/

participants.csv


In [103]:
in_file = "/smartcode/participants/participants.csv"

In [104]:
!head -n2 $in_file

Aljosha,Judmayer,ajudmayer@sba-research.org,1
Nicholas,Stifter,nstifter@sba-research.org,2


In [105]:
!file $in_file # get file format/encoding

/smartcode/participants/participants.csv: ASCII text


In [106]:
!wc -l $in_file # check number of stutend and compare later 

3 /smartcode/participants/participants.csv


In [170]:
students_list = list()
students_count = 0

in_fieldnames = ['firstname', 'lastname','email','matr']

In [171]:
def read_students(path):
    global students_count
    global students_list
    with open(path,'r',encoding='utf-8') as f_input:
    #with open(path,'r',encoding='utf-16') as f_input:
    #with open(path,'r',encoding='latin-1') as f_mail:
        #sreader = csv.DictReader(f_input, fieldnames=in_fieldnames, delimiter='\t',quotechar='\"')
        sreader = csv.DictReader(f_input, fieldnames=in_fieldnames, delimiter=',',quotechar='\"')
        for i,row in enumerate(sreader):
            #print(i,row)
            student_dict = dict()
            for field in in_fieldnames:
                    student_dict[field]=row[field]
            students_list.append(student_dict)
            #print(students_list)
    students_count = i+1
    return students_count

In [172]:
read_students(in_file)

2

In [192]:
students_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'matr': '1'}

In [174]:
students_list[-1]

{'firstname': 'Nicholas',
 'lastname': 'Stifter',
 'email': 'nstifter@sba-research.org',
 'matr': '2'}

In [175]:
students_list

[{'firstname': 'Aljosha',
  'lastname': 'Judmayer',
  'email': 'ajudmayer@sba-research.org',
  'matr': '1'},
 {'firstname': 'Nicholas',
  'lastname': 'Stifter',
  'email': 'nstifter@sba-research.org',
  'matr': '2'}]

In [176]:
nr_students = len(students_list)
nr_students

2

## Create keys

In [177]:
!cat /smartcode/create_accounts.sh 

#!/bin/bash


if [ -z "${1}" ] && [ -z "${PARTICIPANTS+x}" ];
then
    echo "PARTICIPANTS = No number given e.g., '42'"
    exit 2
elif [ ! -z "${1}" ];
then
    export PARTICIPANTS="${1}"
fi

export DATADIR="./datadir/accountmgt"

echo "DATADIR = ${DATADIR}"
echo "PARTICIPANTS = ${PARTICIPANTS}"

mkdir -p "./generated_config/keystore/"

#for i in {0..90}};
for i in $(seq 1 ${PARTICIPANTS});
do 
	echo $i
	bash ./geth_account.sh new
done

cp -r ./datadir/accountmgt/keystore/* ./generated_config/keystore/


In [178]:
!cat /smartcode/geth_account.sh

#!/bin/bash
#
#
# Run with docker:
# $ PWFILE=./passwordfile DATADIR=./datadir/accountmgt/ bash geth_account.sh new

if [ -z "${PWFILE+x}" ];
then
	echo "PWFILE environment variable missing"
	echo "No Password file given using default './passwordfile'"
	export PWFILE='./passwordfile'
fi
echo "PWFILE = ${PWFILE}"

if [ -f "${PWFILE}" ];
then
	echo "using stored password ... "	
else
	pwgen -s -B 32 1 > ${PWFILE}
fi

if [ -z "${DATADIR+x}" ];
then
        echo "DATADIR environment variable missing"
	echo "No Datadir given e.g., '/datadir/alice/'"
	exit 2
fi
echo "DATADIR = ${DATADIR}"

# run container without network
# just to generate and manage keys
docker run \
	--mount type=bind,source=$(pwd),target=/smartenv \
	-it smartenv:latest \
		geth \
		--datadir "${DATADIR}" \
		--nodiscover \
		--maxpeers 0 \
		--ipcdisable \
		--verbosity 6  \
		--password "${PWFILE}" \
		account "$1" "$2" "$3" 



## Assign addresses to users (testaccounts & students)

In [193]:
#### CHANGE ####
 
# path where the raw ethereuem keys are located:
#source_path="./test/keystore/" 
source_path="/smartcode/generated_config/keystore"

# Either run with test users or with all students:
#users_list = copy.deepcopy(testers_list)
users_list = copy.deepcopy(students_list)

#users_list = copy.deepcopy(testers_list)
#users_list.extend(copy.deepcopy(students_list)

#### CHANGE END ####

In [194]:
# print some user as test before assignment
print(len(users_list))
users_list[0]

2


{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'matr': '1'}

In [195]:
!ls /smartcode/generated_config

attach	enodes	keystore


In [196]:
!cd $source_path && pwd && ls | head -n1

/smartcode/generated_config/keystore
UTC--2020-11-17T03-13-03.788368877Z--e96df436579e31e9f8c4b27b3c2e3a911ab94fc2


In [197]:
!cd $source_path && ls ./ | head -n1 | xargs -Irpl cat rpl

{"address":"e96df436579e31e9f8c4b27b3c2e3a911ab94fc2","crypto":{"cipher":"aes-128-ctr","ciphertext":"d18ad320055c601e5a4c6852ff51a01cb41f2bdb0987d18ceea59e41fa4780cf","cipherparams":{"iv":"1353c63310c8f7aeee7858bae03c73bb"},"kdf":"scrypt","kdfparams":{"dklen":32,"n":262144,"p":1,"r":8,"salt":"55c68172867b34811d8624debaedf34b853f85fe76a39f346874d10a97eccb27"},"mac":"064b6acf74ed3048d202537cb71d8d8471f4ffa61b02ae64ab118bdef46dc708"},"id":"74dfa07d-e33b-4882-aedb-d1b4202c317e","version":3}

In [198]:
def read_keypairs(path):
    """read keypairs from keystore
    """
    _keypairs = list()
    # read out keystore folder 
    for root,dirs,files in os.walk(path):
        #print("root = {}".format(root))
        #print("dirs = {}".format(dirs))
        for f in files:
            with open(root + "/" + f,"r",encoding="utf-8") as f_input:
                data = json.loads(f_input.read()) # load all key pair data from file
                data["filename"] = f # add file name to data for later
                _keypairs.append(data)
    return _keypairs   

In [199]:
def assign_addresses(keystore_path,users,attach_path="/smartcode/generated_config/attach"):
    """ map keypairs to users and add info to user dict 
    """
    keypairs = read_keypairs(keystore_path)
    assert len(keypairs) >= len(users)
    
    for u in users:
        kp = keypairs.pop()
        u["account"] = kp["address"]
        #u["keypair"] = kp
        #u["key"] = kp["crypto"]["ciphertext"]
        u["filename"] = kp["filename"]
        
        arg_dest = attach_path + "/" + u["account"]
        !mkdir -p $arg_dest
        arg_src = keystore_path + "/" + kp["filename"]
        !cp $arg_src $arg_dest
        u["attach"] = "/tmp/attach/" + u["account"] + ".tar.gz"
        print(".",end=".")
    return users

In [200]:
attach_path="/smartcode/generated_config/attach"
#!rm -rf attach_path
assign_addresses(source_path,users_list,attach_path)

# print some user as test after assignment
users_list[0]

....

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'matr': '1',
 'account': 'e96df436579e31e9f8c4b27b3c2e3a911ab94fc2',
 'filename': 'UTC--2020-11-17T03-13-03.788368877Z--e96df436579e31e9f8c4b27b3c2e3a911ab94fc2',
 'attach': '/tmp/attach/e96df436579e31e9f8c4b27b3c2e3a911ab94fc2.tar.gz'}

In [204]:
def getBalance(address):
    address = w3.toChecksumAddress(address)
    return w3.fromWei(w3.eth.getBalance(address),'ether')

In [205]:
getBalance(users_list[0]["account"])

0

In [206]:
w3.eth.accounts

['0xe63419B7c0be62A6127923fb70C32A5ca1926a16']

In [207]:
for a in w3.eth.accounts:
    print(a,": ",getBalance(a))

0xe63419B7c0be62A6127923fb70C32A5ca1926a16 :  100000000


In [208]:
w3.eth.defaultAccount = w3.eth.accounts[0]

In [209]:
w3.eth.defaultAccount

'0xe63419B7c0be62A6127923fb70C32A5ca1926a16'

In [210]:
def send_accounts_money(users,amount):
    for u in users:
        account = w3.toChecksumAddress(u["account"])
        u["seedbalance"] = amount
        w3.eth.sendTransaction({"from":w3.eth.defaultAccount,"to":account,"value":amount})
    return

In [211]:
send_accounts_money(users_list,16*10**18)

In [212]:
getBalance(users_list[0]["account"])

Decimal('16')

In [213]:
getBalance(users_list[1]["account"])

Decimal('16')

In [214]:
def check_accounts_money(users,predicate=(lambda a, b: a == b)):
    for u in users:
        u_balance = w3.eth.getBalance(w3.toChecksumAddress(u["account"]))
        u["balance"] = u_balance
        u_seedbalance = u["seedbalance"]
        if predicate(u_balance,u_seedbalance) == False:
            print("Predicate violation for:\n",u)

In [215]:
check_accounts_money(users_list)

In [216]:
check_accounts_money(users_list,predicate=(lambda a, b: a >= b))
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'matr': '1',
 'account': 'e96df436579e31e9f8c4b27b3c2e3a911ab94fc2',
 'filename': 'UTC--2020-11-17T03-13-03.788368877Z--e96df436579e31e9f8c4b27b3c2e3a911ab94fc2',
 'attach': '/tmp/attach/e96df436579e31e9f8c4b27b3c2e3a911ab94fc2.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000}

### Test manual account handling

In [217]:
for a in w3.eth.accounts:
    print(a,": ",getBalance(a))

0xe63419B7c0be62A6127923fb70C32A5ca1926a16 :  99999968


In [219]:
w3.eth.sendTransaction({'from':w3.eth.accounts[0], 
                        'to':"0xEe07C32520Ed77679407ad373A6F10667B5498E2", 
                        'value':10**18})

HexBytes('0x25d03885c21c6e0a16230e9352c6df9932b1ca860820d645d2d5486c3e5b4b37')

### Deployment functions

In [220]:
sys.path.append("/smartcode/util")
import util
from importlib import reload
reload(util)

<module 'util' from '/smartcode/notebooks/challenges/util/__init__.py'>

In [221]:
def forall_compile_and_deploy(users,path,contract_name,value,force_argument=None,argument2=None):
    print("Started for {} users:".format(len(users)))
    for u in users:
        if force_argument is not None:
            print("*",end="")
            #print("Force argument: {}".format(force_argument))
            u_addr = force_argument
        else:
            u_addr = w3.toChecksumAddress(u["account"])
            
        if argument2 is not None and argument2 is True:
            print("~",end="")
            #print("Double sha3 argument2: {}".format(argument2))
            tx_hash = util.compile_and_deploy_contract(path,
                                                   patch_api=False,
                                                   concise=True,
                                                   argument=u_addr,
                                                   argument2=w3.toInt(hexstr=w3.sha3(text=w3.sha3(text=u_addr).hex()).hex()),
                                                   wait=False,
                                                   value=value)
        elif argument2 is not None and isinstance(argument2,type(str())):
            print("+",end="")
            #print("Custom argument2: {}".format(argument2))
            tx_hash = util.compile_and_deploy_contract(path,
                                                   patch_api=False,
                                                   concise=True,
                                                   argument=u_addr,
                                                   argument2=argument2,
                                                   wait=False,
                                                   value=value)
        else:
            print(".",end="")
            tx_hash = util.compile_and_deploy_contract(path,
                                                   patch_api=False,
                                                   concise=True,
                                                   argument=u_addr,
                                                   wait=False,
                                                   value=value)
        # returns HexBytes
        # import hexbytes
        # help(hexbytes.main)
        u[contract_name + "_tx"] = tx_hash.hex()
        u[contract_name + "_seedvalue"] = value
        
    for u in users:
        tx_receipt = w3.eth.waitForTransactionReceipt(u[contract_name + "_tx"])
        u[contract_name + "_addr"] = tx_receipt['contractAddress']
        print("-",end="")
        
    return users

In [222]:
def forall_check_contract(users,c_path,c_name,predicate=(lambda a, b: a == b)):
    # Check for all users if the contract is initialized to the respective user
    # and that the balance is the seed value
    print("Started for {} users:".format(len(users)))
    for u in users:
        c_abi = util.compile_contract_with_libs(compiler_path="solc",src_path=c_path)["abi"]
        u_c_addr = u[c_name + "_addr"]
        u_c_instance = util.get_contract_instance(u_c_addr,c_abi,concise=False)
    
        # check owner
        #print(u_c_instance.functions.getStudent().call())
        #print(w3.toChecksumAddress(u["account"]))
        assert u_c_instance.functions.getStudent().call() == w3.toChecksumAddress(u["account"])   
        # check balance
        u_c_seedvalue = u[c_name + "_seedvalue"]
        u_c_balance = w3.eth.getBalance(u_c_addr)   
        u[c_name + "_balance"] = u_c_balance
        if predicate(u_c_balance,u_c_seedvalue) == False:
            print("Predicate violation for:\n",u)
        print(".",end="")
    return users

In [223]:
def copy_challenge_metadata(users,challenge_path,challenge_name,attach_path="./attach"):
    # copy challenge metadata to individual user folders helper function
    for d in os.listdir(attach_path):
        #print(d)
        if os.path.isdir(os.path.join(attach_path, d)):
            #print(d)
            arg_dest_dir = attach_path + "/" + d + "/" + challenge_name
            print(arg_dest_dir)
            !mkdir -p $arg_dest_dir
            !cp -r $challenge_path $arg_dest_dir

### Challange: notawallet

In [224]:
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'matr': '1',
 'account': 'e96df436579e31e9f8c4b27b3c2e3a911ab94fc2',
 'filename': 'UTC--2020-11-17T03-13-03.788368877Z--e96df436579e31e9f8c4b27b3c2e3a911ab94fc2',
 'attach': '/tmp/attach/e96df436579e31e9f8c4b27b3c2e3a911ab94fc2.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000}

In [225]:
users_list[1]

{'firstname': 'Nicholas',
 'lastname': 'Stifter',
 'email': 'nstifter@sba-research.org',
 'matr': '2',
 'account': '0b82262b7e093adb79d01189f18d39369a1871b7',
 'filename': 'UTC--2020-11-17T03-19-13.937143112Z--0b82262b7e093adb79d01189f18d39369a1871b7',
 'attach': '/tmp/attach/0b82262b7e093adb79d01189f18d39369a1871b7.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000}

In [226]:
!solc --version

solc, the solidity compiler commandline interface
Version: 0.5.4+commit.9549d8ff.Linux.g++


In [227]:
!solc --overwrite -o "/smartcode/challenges/notawallet/publish/" --abi --bin "/smartcode/challenges/notawallet/publish/NotAWallet.sol"

Compiler run successful. Artifact(s) can be found in directory /smartcode/challenges/notawallet/publish/.


In [234]:
## Connections refresh maybe needed
# w3=util.connect(host="172.18.0.5",port="8544",poa=True) 
assert w3.isConnected()

In [231]:
path="/smartcode/challenges/notawallet/publish/NotAWallet.sol"
challange_name="notawallet"
seedvalue=10*10**18

forall_compile_and_deploy(users_list,path,challange_name,seedvalue)
users_list[0]

Started for 2 users:
..--

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'matr': '1',
 'account': 'e96df436579e31e9f8c4b27b3c2e3a911ab94fc2',
 'filename': 'UTC--2020-11-17T03-13-03.788368877Z--e96df436579e31e9f8c4b27b3c2e3a911ab94fc2',
 'attach': '/tmp/attach/e96df436579e31e9f8c4b27b3c2e3a911ab94fc2.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0xc781ce8d743cda6032f50d2040564bb13e19d008ef30cc713effc7e5004c4b63',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0xc59fD2e2fA3c2DFa9c6923bdDD84AC9fA00cFEF8'}

In [241]:
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'matr': '1',
 'account': 'e96df436579e31e9f8c4b27b3c2e3a911ab94fc2',
 'filename': 'UTC--2020-11-17T03-13-03.788368877Z--e96df436579e31e9f8c4b27b3c2e3a911ab94fc2',
 'attach': '/tmp/attach/e96df436579e31e9f8c4b27b3c2e3a911ab94fc2.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0xc781ce8d743cda6032f50d2040564bb13e19d008ef30cc713effc7e5004c4b63',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0xc59fD2e2fA3c2DFa9c6923bdDD84AC9fA00cFEF8'}

In [236]:
!solc --version

solc, the solidity compiler commandline interface
Version: 0.5.4+commit.9549d8ff.Linux.g++


In [237]:
c_abi = util.compile_contract_with_libs(compiler_path="solc",src_path="/smartcode/challenges/notawallet/publish/NotAWallet.sol")["abi"]
c_abi

'[{"constant":false,"inputs":[{"name":"oldowner","type":"address"}],"name":"removeOwner","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"withdraw","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[{"name":"testowner","type":"address"}],"name":"isOwner","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"student","outputs":[{"name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"newowner","type":"address"}],"name":"addOwner","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"owner","outputs":[{"name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name"

In [238]:
notawallet_addr = users_list[0]["notawallet_addr"]
notawallet_addr

'0xc59fD2e2fA3c2DFa9c6923bdDD84AC9fA00cFEF8'

In [239]:
u_c_instance = util.get_contract_instance(notawallet_addr,c_abi,concise=False)

In [240]:
assert u_c_instance.functions.getStudent().call() == w3.toChecksumAddress(users_list[0]["account"])
u_c_instance.functions.getStudent().call()

'0xe96Df436579E31E9f8c4B27B3C2e3a911aB94fc2'

In [242]:
path="/smartcode/challenges/notawallet/publish/NotAWallet.sol"
challange_name="notawallet"
forall_check_contract(users_list,path,challange_name)
#forall_check_contract(users_list,path,challange_name,predicate=(lambda a, b: a < b))
#forall_check_contract(users_list,path,challange_name,predicate=(lambda a, b: a == 10**18))
users_list[0]

Started for 2 users:
..

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'matr': '1',
 'account': 'e96df436579e31e9f8c4b27b3c2e3a911ab94fc2',
 'filename': 'UTC--2020-11-17T03-13-03.788368877Z--e96df436579e31e9f8c4b27b3c2e3a911ab94fc2',
 'attach': '/tmp/attach/e96df436579e31e9f8c4b27b3c2e3a911ab94fc2.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0xc781ce8d743cda6032f50d2040564bb13e19d008ef30cc713effc7e5004c4b63',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0xc59fD2e2fA3c2DFa9c6923bdDD84AC9fA00cFEF8',
 'notawallet_balance': 10000000000000000000}

In [243]:
# copy challenge metadata
challenge_path="/smartcode/challenges/notawallet/publish/*"
attach_path="/smartcode/generated_config/attach"
challenge_name="notawallet"
copy_challenge_metadata(users_list,challenge_path,challenge_name,attach_path)

/smartcode/generated_config/attach/0b82262b7e093adb79d01189f18d39369a1871b7/notawallet
/smartcode/generated_config/attach/e96df436579e31e9f8c4b27b3c2e3a911ab94fc2/notawallet


## Challenge: badparity

In [245]:
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'matr': '1',
 'account': 'e96df436579e31e9f8c4b27b3c2e3a911ab94fc2',
 'filename': 'UTC--2020-11-17T03-13-03.788368877Z--e96df436579e31e9f8c4b27b3c2e3a911ab94fc2',
 'attach': '/tmp/attach/e96df436579e31e9f8c4b27b3c2e3a911ab94fc2.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0xc781ce8d743cda6032f50d2040564bb13e19d008ef30cc713effc7e5004c4b63',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0xc59fD2e2fA3c2DFa9c6923bdDD84AC9fA00cFEF8',
 'notawallet_balance': 10000000000000000000}

In [246]:
!cat "/smartcode/challenges/badparity/publish/Wallet.sol"

pragma solidity ^0.5.4;

contract Wallet {
    address walletLibrary;
    address payable owner;
    address payable student;

    event LogValue(uint256 exitcode,uint256 amount);

    // constructor, called once when this contract is created 
    constructor(address payable _student, address lib) public payable {
        student = _student;  
        walletLibrary = lib; // hardcode lib address at deploy time
        // init the owner with the respective lib contract
        walletLibrary.delegatecall(abi.encodeWithSignature("initWallet(address)", msg.sender));
    }

    function getOwner() public view returns (address payable) {
        return owner;
    } 

    function getWalletLibrary() public view returns (address) {
        return walletLibrary;
    }

    function withdraw(uint256 amount) public returns (bool) {
        (bool success, bytes memory data) = walletLibrary.delegatecall(abi.encodeWithSignature("withdraw(uint256)", amount));
        if ( success ) {
            emit

In [247]:
!solc --overwrite -o "/smartcode/challenges/badparity/publish/" --abi --bin "/smartcode/challenges/badparity/publish/Wallet.sol"

/smartcode/challenges/badparity/publish/Wallet.sol:15:9: Warning: Return value of low-level calls not used.
        walletLibrary.delegatecall(abi.encodeWithSignature("initWallet(address)", msg.sender));
        ^------------------------------------------------------------------------------------^
/smartcode/challenges/badparity/publish/Wallet.sol:49:9: Warning: Return value of low-level calls not used.
        walletLibrary.delegatecall(msg.data);
        ^----------------------------------^
/smartcode/challenges/badparity/publish/Wallet.sol:27:24: Warning: Unused local variable.
        (bool success, bytes memory data) = walletLibrary.delegatecall(abi.encodeWithSignature("withdraw(uint256)", amount));
                       ^---------------^
/smartcode/challenges/badparity/publish/Wallet.sol:37:24: Warning: Unused local variable.
        (bool success, bytes memory data) = walletLibrary.delegatecall(abi.encodeWithSignature("changeOwner(address)", new_owner));
                       

In [248]:
!solc --overwrite -o "/smartcode/challenges/badparity/publish/" --abi --bin "/smartcode/challenges/badparity/publish/WalletLibrary.sol"

Compiler run successful. Artifact(s) can be found in directory /smartcode/challenges/badparity/publish/.


In [249]:
# compile and deploy lib once for all users
path="/smartcode/challenges/badparity/publish/WalletLibrary.sol"

lib = util.compile_and_deploy_contract(path,
                                 patch_api=False,
                                 concise=False)
lib.address

'0xa1e729D031760D9B2741968933253a8a8F905FFf'

In [250]:
path="/smartcode/challenges/badparity/publish/Wallet.sol"
challange_name="badparity"
seedvalue=30*10**18

forall_compile_and_deploy(users_list,path,challange_name,seedvalue,argument2=lib.address)

for u in users_list:
    u[challange_name + "_lib"] = lib.address

users_list[0]

Started for 2 users:
++--

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'matr': '1',
 'account': 'e96df436579e31e9f8c4b27b3c2e3a911ab94fc2',
 'filename': 'UTC--2020-11-17T03-13-03.788368877Z--e96df436579e31e9f8c4b27b3c2e3a911ab94fc2',
 'attach': '/tmp/attach/e96df436579e31e9f8c4b27b3c2e3a911ab94fc2.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0xc781ce8d743cda6032f50d2040564bb13e19d008ef30cc713effc7e5004c4b63',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0xc59fD2e2fA3c2DFa9c6923bdDD84AC9fA00cFEF8',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0x57bf907cc0218404c73175ea264fa1006ca90dc4fc7891305fe2e18b9d64ad97',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0xe1FCa653e6a20aFe27eC2BF6735387F5a5909284',
 'badparity_lib': '0xa1e729D031760D9B2741968933253a8a8F905FFf'}

In [251]:
# spot check of one contract
c_abi = util.compile_contract_with_libs("solc",path)["abi"]
u_c_addr = users_list[0][challange_name + "_addr"]
u_c_instance = util.get_contract_instance(u_c_addr,c_abi,concise=True)

assert users_list[0][challange_name + "_lib"] == lib.address
    
# check lib
assert str(u_c_instance.getWalletLibrary()).lower() == str(lib.address).lower()
assert str(u_c_instance.getOwner()).lower() == str(w3.eth.accounts[0]).lower()

# check balance
assert w3.eth.getBalance(u_c_addr) == 30*10**18  

In [252]:
u_c_instance.getOwner()

'0xe63419B7c0be62A6127923fb70C32A5ca1926a16'

In [253]:
u_c_instance.getStudent()

'0xe96Df436579E31E9f8c4B27B3C2e3a911aB94fc2'

In [254]:
w3.parity.personal.listAccounts()

['0xe63419B7c0be62A6127923fb70C32A5ca1926a16']

In [255]:
# copy challenge metadata
challenge_path="/smartcode/challenges/badparity/publish/*"
attach_path="/smartcode/generated_config/attach"
challenge_name="badparity"
copy_challenge_metadata(users_list,challenge_path,challenge_name,attach_path)

/smartcode/generated_config/attach/0b82262b7e093adb79d01189f18d39369a1871b7/badparity
/smartcode/generated_config/attach/e96df436579e31e9f8c4b27b3c2e3a911ab94fc2/badparity


## Challenge: Daodown

In [257]:
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'matr': '1',
 'account': 'e96df436579e31e9f8c4b27b3c2e3a911ab94fc2',
 'filename': 'UTC--2020-11-17T03-13-03.788368877Z--e96df436579e31e9f8c4b27b3c2e3a911ab94fc2',
 'attach': '/tmp/attach/e96df436579e31e9f8c4b27b3c2e3a911ab94fc2.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0xc781ce8d743cda6032f50d2040564bb13e19d008ef30cc713effc7e5004c4b63',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0xc59fD2e2fA3c2DFa9c6923bdDD84AC9fA00cFEF8',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0x57bf907cc0218404c73175ea264fa1006ca90dc4fc7891305fe2e18b9d64ad97',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0xe1FCa653e6a20aFe27eC2BF6735387F5a5909284',
 'badparity_lib': '0xa1e729D031760D9B2741968933253a8a8F905FFf'}

In [258]:
!solc --overwrite -o "/smartcode/challenges/daodown/publish/" --abi --bin "/smartcode/challenges/daodown/publish/EDao.sol"

Compiler run successful. Artifact(s) can be found in directory /smartcode/challenges/daodown/publish/.


In [259]:
path="/smartcode/challenges/daodown/publish/EDao.sol"
challange_name="daodown"
seedvalue=30*10**18

forall_compile_and_deploy(users_list,path,challange_name,seedvalue)
users_list[0]

Started for 2 users:
..--

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'matr': '1',
 'account': 'e96df436579e31e9f8c4b27b3c2e3a911ab94fc2',
 'filename': 'UTC--2020-11-17T03-13-03.788368877Z--e96df436579e31e9f8c4b27b3c2e3a911ab94fc2',
 'attach': '/tmp/attach/e96df436579e31e9f8c4b27b3c2e3a911ab94fc2.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0xc781ce8d743cda6032f50d2040564bb13e19d008ef30cc713effc7e5004c4b63',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0xc59fD2e2fA3c2DFa9c6923bdDD84AC9fA00cFEF8',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0x57bf907cc0218404c73175ea264fa1006ca90dc4fc7891305fe2e18b9d64ad97',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0xe1FCa653e6a20aFe27eC2BF6735387F5a5909284',
 'badparity_lib': '0xa1e729D031760D9B2741968933253a8a8F905FFf',
 'daodown_tx': '0x2ec2d8ff28c16193cfc3acd8d32466b71d0fd33d343abfcb638f2c3a9d911587',
 'daodo

In [260]:
w3.eth.blockNumber

18888

In [262]:
w3.eth.getBlock(w3.eth.blockNumber)["gasUsed"]

1852514

In [261]:
w3.eth.getBlock(w3.eth.blockNumber-3)["gasUsed"]

0

In [263]:
path="/smartcode/challenges/daodown/publish/EDao.sol"
challange_name="daodown"
forall_check_contract(users_list,path,challange_name)
#forall_check_contract(users_list,path,challange_name,predicate=(lambda a, b: a < b))
#forall_check_contract(users_list,path,challange_name,predicate=(lambda a, b: a == 10**18))
users_list[0]

Started for 2 users:
..

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'matr': '1',
 'account': 'e96df436579e31e9f8c4b27b3c2e3a911ab94fc2',
 'filename': 'UTC--2020-11-17T03-13-03.788368877Z--e96df436579e31e9f8c4b27b3c2e3a911ab94fc2',
 'attach': '/tmp/attach/e96df436579e31e9f8c4b27b3c2e3a911ab94fc2.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0xc781ce8d743cda6032f50d2040564bb13e19d008ef30cc713effc7e5004c4b63',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0xc59fD2e2fA3c2DFa9c6923bdDD84AC9fA00cFEF8',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0x57bf907cc0218404c73175ea264fa1006ca90dc4fc7891305fe2e18b9d64ad97',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0xe1FCa653e6a20aFe27eC2BF6735387F5a5909284',
 'badparity_lib': '0xa1e729D031760D9B2741968933253a8a8F905FFf',
 'daodown_tx': '0x2ec2d8ff28c16193cfc3acd8d32466b71d0fd33d343abfcb638f2c3a9d911587',
 'daodo

In [264]:
# copy challenge metadata
challenge_path="/smartcode/challenges/daodown/publish/*"
attach_path="/smartcode/generated_config/attach"
challenge_name="daodown"
copy_challenge_metadata(users_list,challenge_path,challenge_name,attach_path)

/smartcode/generated_config/attach/0b82262b7e093adb79d01189f18d39369a1871b7/daodown
/smartcode/generated_config/attach/e96df436579e31e9f8c4b27b3c2e3a911ab94fc2/daodown


## Challenge: Faildice

In [265]:
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'matr': '1',
 'account': 'e96df436579e31e9f8c4b27b3c2e3a911ab94fc2',
 'filename': 'UTC--2020-11-17T03-13-03.788368877Z--e96df436579e31e9f8c4b27b3c2e3a911ab94fc2',
 'attach': '/tmp/attach/e96df436579e31e9f8c4b27b3c2e3a911ab94fc2.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0xc781ce8d743cda6032f50d2040564bb13e19d008ef30cc713effc7e5004c4b63',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0xc59fD2e2fA3c2DFa9c6923bdDD84AC9fA00cFEF8',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0x57bf907cc0218404c73175ea264fa1006ca90dc4fc7891305fe2e18b9d64ad97',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0xe1FCa653e6a20aFe27eC2BF6735387F5a5909284',
 'badparity_lib': '0xa1e729D031760D9B2741968933253a8a8F905FFf',
 'daodown_tx': '0x2ec2d8ff28c16193cfc3acd8d32466b71d0fd33d343abfcb638f2c3a9d911587',
 'daodo

In [267]:
!ls -la /usr/local/bin/solc*

-rwxr-xr-x 1 root root  6284040 Nov 13 16:11 /usr/local/bin/solc
-rwxr-xr-x 1 root root  6284040 Feb 12  2019 /usr/local/bin/solc_5.4
-rwxr-xr-x 1 root root 11192752 Oct 19 16:17 /usr/local/bin/solc_7.4


In [269]:
!/usr/local/bin/solc --version

solc, the solidity compiler commandline interface
Version: 0.5.4+commit.9549d8ff.Linux.g++


In [271]:
!solc --overwrite -o "/smartcode/challenges/faildice/publish/" --abi --bin "/smartcode/challenges/faildice/publish/SatoshiFailDice.sol"

Compiler run successful. Artifact(s) can be found in directory /smartcode/challenges/faildice/publish/.


In [272]:
path="/smartcode/challenges/faildice/publish/SatoshiFailDice.sol"
challange_name="faildice"
seedvalue=30*10**18

forall_compile_and_deploy(users_list,path,challange_name,seedvalue,argument2=True)
users_list[0]

Started for 2 users:
~~--

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'matr': '1',
 'account': 'e96df436579e31e9f8c4b27b3c2e3a911ab94fc2',
 'filename': 'UTC--2020-11-17T03-13-03.788368877Z--e96df436579e31e9f8c4b27b3c2e3a911ab94fc2',
 'attach': '/tmp/attach/e96df436579e31e9f8c4b27b3c2e3a911ab94fc2.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0xc781ce8d743cda6032f50d2040564bb13e19d008ef30cc713effc7e5004c4b63',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0xc59fD2e2fA3c2DFa9c6923bdDD84AC9fA00cFEF8',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0x57bf907cc0218404c73175ea264fa1006ca90dc4fc7891305fe2e18b9d64ad97',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0xe1FCa653e6a20aFe27eC2BF6735387F5a5909284',
 'badparity_lib': '0xa1e729D031760D9B2741968933253a8a8F905FFf',
 'daodown_tx': '0x2ec2d8ff28c16193cfc3acd8d32466b71d0fd33d343abfcb638f2c3a9d911587',
 'daodo

In [273]:
path="/smartcode/challenges/faildice/publish/SatoshiFailDice.sol"
challange_name="faildice"
forall_check_contract(users_list,path,challange_name)
#forall_check_contract(users_list,path,challange_name,predicate=(lambda a, b: a < b))
#forall_check_contract(users_list,path,challange_name,predicate=(lambda a, b: a == 10**18))
users_list[0]

Started for 2 users:
..

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'matr': '1',
 'account': 'e96df436579e31e9f8c4b27b3c2e3a911ab94fc2',
 'filename': 'UTC--2020-11-17T03-13-03.788368877Z--e96df436579e31e9f8c4b27b3c2e3a911ab94fc2',
 'attach': '/tmp/attach/e96df436579e31e9f8c4b27b3c2e3a911ab94fc2.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0xc781ce8d743cda6032f50d2040564bb13e19d008ef30cc713effc7e5004c4b63',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0xc59fD2e2fA3c2DFa9c6923bdDD84AC9fA00cFEF8',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0x57bf907cc0218404c73175ea264fa1006ca90dc4fc7891305fe2e18b9d64ad97',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0xe1FCa653e6a20aFe27eC2BF6735387F5a5909284',
 'badparity_lib': '0xa1e729D031760D9B2741968933253a8a8F905FFf',
 'daodown_tx': '0x2ec2d8ff28c16193cfc3acd8d32466b71d0fd33d343abfcb638f2c3a9d911587',
 'daodo

In [274]:
# copy challenge metadata
challenge_path="/smartcode/challenges/faildice/publish/*"
attach_path="/smartcode/generated_config/attach"
challenge_name="faildice"
copy_challenge_metadata(users_list,challenge_path,challenge_name,attach_path)

/smartcode/generated_config/attach/0b82262b7e093adb79d01189f18d39369a1871b7/faildice
/smartcode/generated_config/attach/e96df436579e31e9f8c4b27b3c2e3a911ab94fc2/faildice


## Write Students

In [275]:
import json

def dump_users(path,users):
    with open(path,'w') as f_output:
        json.dump(users,f_output)

In [276]:
out_file_json = "/smartcode/generated_config/students.json"
dump_users(out_file_json,users_list)
len(users_list)

2

In [277]:
import csv

def write_students(path,
                   out_fieldnames=['firstname', 'lastname','matr','email','cc','attach','contract','account'],
                   cc_mail="ajudmayer@sba-research.org"):
    with open(path,'w',encoding='utf-8') as f_output:
    #with open(path,'w',encoding='latin-1') as f_output:
        writer = csv.DictWriter(f_output, 
                                fieldnames=out_fieldnames, 
                                delimiter=',',
                                quotechar='\"',
                                extrasaction='ignore')
        writer.writeheader()
        for i,row in enumerate(users_list):
            row["cc"]=cc_mail
            #print(row)
            writer.writerow(row)
    return i+1

In [278]:
out_file = "/smartcode/generated_config/students.csv"
out_fieldnames = ['matr','email','cc','attach','account']
#out_fieldnames = ['firstname', 'lastname','matr','email','cc','attach','contract','account']
#out_fieldnames = ['firstname', 
#                  'lastname',
#                  'matr',
#                  'email',
#                  'cc',
#                  'attach',
#                  'account',
#                  'notawallet_addr',
#                 ]
cc_mail = "ajudmayer@sba-research.org"
write_students(out_file,out_fieldnames,cc_mail)

2

In [279]:
!head -n2 $out_file

In [280]:
!wc -l $out_file

3 /smartcode/generated_config/students.csv


## Prepare mail

In [283]:
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'matr': '1',
 'account': 'e96df436579e31e9f8c4b27b3c2e3a911ab94fc2',
 'filename': 'UTC--2020-11-17T03-13-03.788368877Z--e96df436579e31e9f8c4b27b3c2e3a911ab94fc2',
 'attach': '/tmp/attach/e96df436579e31e9f8c4b27b3c2e3a911ab94fc2.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0xc781ce8d743cda6032f50d2040564bb13e19d008ef30cc713effc7e5004c4b63',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0xc59fD2e2fA3c2DFa9c6923bdDD84AC9fA00cFEF8',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0x57bf907cc0218404c73175ea264fa1006ca90dc4fc7891305fe2e18b9d64ad97',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0xe1FCa653e6a20aFe27eC2BF6735387F5a5909284',
 'badparity_lib': '0xa1e729D031760D9B2741968933253a8a8F905FFf',
 'daodown_tx': '0x2ec2d8ff28c16193cfc3acd8d32466b71d0fd33d343abfcb638f2c3a9d911587',
 'daodo

In [284]:
import os

def dump_addresses(users,attach_path="/smartcode/generated_config/attach"):
    for u in users:
        arg_dest = attach_path + "/" + u["account"] + "/" + "addresses"
        ch_string = "notawallet_addr = " + u["notawallet_addr"]
        !echo $ch_string >> $arg_dest
        
        ch_string = "badparity_addr = " + u["badparity_addr"]
        !echo $ch_string >> $arg_dest    
        
        ch_string = "daodown_addr = " + u["daodown_addr"]
        !echo $ch_string >> $arg_dest
        
        ch_string = "faildice_addr = " + u["faildice_addr"]
        !echo $ch_string >> $arg_dest
        
        print(".",end="")

In [285]:
# write all addresses to folders 
dump_addresses(users_list,attach_path="/smartcode/generated_config/attach")

..

In [287]:
!cp /smartcode/passwordfile /smartcode/general_info/

In [288]:
# copy general information i.e., datadir to individual folders
source_path="/smartcode/general_info/*"
attach_path="/smartcode/generated_config/attach"
challenge_name=""
copy_challenge_metadata(users_list,source_path,challenge_name,attach_path)

/smartcode/generated_config/attach/0b82262b7e093adb79d01189f18d39369a1871b7/
/smartcode/generated_config/attach/e96df436579e31e9f8c4b27b3c2e3a911ab94fc2/


In [289]:
# create archives
import os

def create_user_archives(users,attach_path="/smartcode/generated_config/attach",attach_tgz="/smartcode/generated_config/attach_tgz"):
    !mkdir -p $attach_tgz
    for d in os.listdir(attach_path):
        #print(d)
        if os.path.isdir(os.path.join(attach_path, d)):
            #print(d)
            arg_src_dir = attach_path + "/" + d 
            arg_dst_dir = attach_tgz + "/" + d + ".tar.gz"
            print(arg_dst_dir)
            !tar -cvzf $arg_dst_dir -C $arg_src_dir .

In [290]:
attach_path="/smartcode/generated_config/attach"
attach_tgz="/smartcode/generated_config/attach_tgz"
create_user_archives(users_list,attach_path,attach_tgz)

/smartcode/generated_config/attach_tgz/0b82262b7e093adb79d01189f18d39369a1871b7.tar.gz
./
./README.md
./addresses
./notawallet/
./notawallet/README.md
./notawallet/NotAWallet.sol
./notawallet/NotAWallet.abi
./notawallet/NotAWallet.bin
./daodown/
./daodown/EDao.bin
./daodown/README.md
./daodown/EDao.sol
./daodown/EDao.abi
./badparity/
./badparity/WalletLibrary.bin
./badparity/WalletLibrary.sol
./badparity/Wallet.abi
./badparity/README.md
./badparity/WalletLibrary.abi
./badparity/Wallet.sol
./badparity/Wallet.bin
./faildice/
./faildice/README.md
./faildice/SatoshiFailDice.bin
./faildice/SatoshiFailDice.abi
./faildice/SatoshiFailDice.sol
./UTC--2020-11-17T03-19-13.937143112Z--0b82262b7e093adb79d01189f18d39369a1871b7
./passwordfile
/smartcode/generated_config/attach_tgz/e96df436579e31e9f8c4b27b3c2e3a911ab94fc2.tar.gz
./
./README.md
./addresses
./notawallet/
./notawallet/README.md
./notawallet/NotAWallet.sol
./notawallet/NotAWallet.abi
./notawallet/NotAWallet.bin
./daodown/
./daodown/EDao.b